In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import joblib
from sklearn.metrics import mean_squared_error, r2_score

/tmp/ipykernel_887769/2481740200.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
random_state = 20
threshold = 1 # Accuracy scoring threshold

In [3]:
# Import data and preprocess
df = pd.read_csv("../test_data/new leaky wave/S11_Data_combined.csv")
df = df.drop(df[df['dB(S(1,1)) []'] > 0].index) # Remove all rows with positive s11

In [4]:
df

,cpw_in [mm],feed_l [mm],patch_l [mm],cpw_g [mm],Feed_W [mm],ground_w [mm],patch_ground_w [mm],patch_w [mm],Freq [GHz],"dB(S(1,1)) []"
0,1.5,3.25,3.25,0.18,1.0,1.0,1.0,4.75,11.00,-1.455338
1,1.5,3.25,3.25,0.18,1.0,1.0,1.0,4.75,11.09,-1.495640
2,1.5,3.25,3.25,0.18,1.0,1.0,1.0,4.75,11.18,-1.566487
3,1.5,3.25,3.25,0.18,1.0,1.0,1.0,4.75,11.27,-1.667701
4,1.5,3.25,3.25,0.18,1.0,1.0,1.0,4.75,11.36,-1.799854
...,...,...,...,...,...,...,...,...,...,...
23730,2.0,3.75,3.50,0.18,1.0,1.5,1.5,5.00,19.64,-3.573784
23731,2.0,3.75,3.50,0.18,1.0,1.5,1.5,5.00,19.73,-2.925513
23732,2.0,3.75,3.50,0.18,1.0,1.5,1.5,5.00,19.82,-2.792273
23733,2.0,3.75,3.50,0.18,1.0,1.5,1.5,5.00,19.91,-3.092972


In [5]:
# Split into x and y
input_x = df.drop(columns=['dB(S(1,1)) []'], axis=1)
input_y = df[['dB(S(1,1)) []']]

In [6]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(input_x, input_y, random_state=random_state)

In [7]:
model = XGBRegressor(n_estimators=1100, min_child_weight=2, max_depth=10, learning_rate=0.2)
scaler = StandardScaler() # Initialize scaler
pipeline = Pipeline(steps=[('normalize', scaler), ('model', model)]) # Create pipeline with scaler and model

In [8]:
%%time
# Train and predict the pipeline
pipeline_fit = pipeline.fit(X_train, y_train)

CPU times: user 40.6 s, sys: 624 ms, total: 41.2 s
Wall time: 2.22 s


In [9]:
%%time
predictions = pipeline_fit.predict(X_test)

CPU times: user 378 ms, sys: 0 ns, total: 378 ms
Wall time: 22.1 ms


In [10]:
# Check if predicted value is threshold amount above or below actual value
def is_in_threshold(actual, pred):
    return pred <= actual + threshold and pred >= actual - threshold

In [11]:
# Create new boolean column that shows if the test and prediction values are the same
results = X_test.copy()
def create_tf_column(results):
    return results.apply(lambda x: is_in_threshold(x['y_test'], x['predictions']), axis=1)

In [12]:
# Calculate accuracy of model by number of predictions that are within threshold value above or below the test value for each row
def get_score(X_test, y_test, clf_dt):
    predictions = clf_dt.predict(X_test)
    dataframe = pd.DataFrame(X_test.copy(), columns=input_x.columns)
    dataframe['y_test'] = y_test.values
    dataframe['predictions'] = predictions
    return create_tf_column(dataframe).value_counts().get(True) / dataframe.shape[0]

In [13]:
X_test

,cpw_in [mm],feed_l [mm],patch_l [mm],cpw_g [mm],Feed_W [mm],ground_w [mm],patch_ground_w [mm],patch_w [mm],Freq [GHz]
20014,2.00,3.75,3.50,0.18,1.0,1.00,1.00,5.00,12.44
11297,1.75,3.75,4.25,0.18,1.0,1.00,1.00,4.75,18.74
3335,2.25,3.50,3.50,0.18,1.0,1.00,1.00,4.75,11.18
5061,1.50,3.25,3.75,0.18,1.0,1.00,1.00,4.75,11.99
2513,2.50,4.25,3.25,0.18,1.0,1.00,1.00,4.75,19.01
...,...,...,...,...,...,...,...,...,...
3188,1.75,3.50,3.50,0.18,1.0,1.00,1.00,4.75,16.13
17308,2.00,3.75,3.50,0.18,1.0,1.25,0.50,5.00,14.33
18208,2.00,3.75,3.50,0.18,1.0,0.75,0.75,5.00,13.52
1867,2.25,4.00,3.25,0.18,1.0,1.00,1.00,4.75,15.41


In [14]:
print(f"Score within +-{threshold}: {get_score(X_test, y_test, pipeline)}")
print("RMSE:", mean_squared_error(y_test, predictions, squared=False))
print("R^2:", r2_score(y_test, predictions))

Score within +-1: 0.671216717222784
RMSE: 2.6936414333229854
R^2: 0.7667957820535474


In [15]:
joblib.dump(pipeline, "antenna_model.pkl")

['antenna_model.pkl']